In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import html5lib

path = 'example.html'
with open(path, 'rt') as myfile:
    data = myfile.read().replace("<br>", '\n')


df = pd.read_html(data)

In [2]:
# Since the older version cape uses a much simpler HTML tree, this is wayy easier to clean
pd.read_html("07capessample.html")[0]

,0,1,2,3,4,5
0,VIS,VIS 60,(A),"Michaelis, Danielle Ruth","Michaelis, Danielle Ruth",SU07
1,NaN,NaN,NaN,Enrollment: 19,Questionnaire's Returned: 18,SelfCAPE


In [3]:
df[3]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,NaN,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,0,0.00,NaN


In [4]:
soup = BeautifulSoup(open("example.html", "r"))

In [5]:
pd.read_html("example.html")[2]

,A,B,C,D,F,P,NP
0,43,7,2,0,3,9,2
1,65 %,11 %,3 %,0 %,5 %,14 %,3 %


In [6]:
import re

In [7]:
# seperate question, column head, response for each question using dictionaries --> then create dataframe to fit them

In [8]:
soup.find("span", {"id": "ContentPlaceHolder1_lblCourseDescription"}).text.split("(")[0].strip()

'AAS 10'

In [9]:
def question_collector(bs4obj):
    question_text = bs4obj.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_.*")})
    dct_key = []
    general_info_dct = dict(zip(["Course", "Instructor", "Qtr/Yr", "Enrollment", "CAPEs Returned"], 
                                           [[i.text] for i in bs4obj.find_all(
                                               "span", {"id": re.compile("ContentPlaceHolder1_lbl.*"), 
                                                        "style": "font-weight:bold;"})]))
    general_info_dct["Question"] = "General Info"
    return_dct = {"General Info": {"General Info": general_info_dct}}
    counter = 0
    for qt in question_text:
        if qt.text.split()[0].isupper() and len(qt.text.split()[0]) > 1:
            dct_key.append(qt.text)
            return_dct[qt.text] = {}
            counter += 1
        else:
            if len(soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_" 
                                                                + str(counter)+"_.*")})) != 0:
                col = dict(zip([i.text for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_" 
                                                                    + str(counter)+"_.*")})],
                                [[i.get_text(separator=" ").split(" ")[0]] for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoices_" 
                                                                    + str(counter)+"_.*")})]))
                col_copy = col.copy()
                col_copy.update({"Question": qt.text})
                return_dct[dct_key[-1]].update({qt.text: col_copy})
                counter += 1
            else:
                col = dict(zip(col.keys(), [[i.get_text(separator=" ").split(" ")[0]] for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoices_" 
                                                                    + str(counter)+"_.*")})]))
                col_copy = col.copy()
                col_copy.update({"Question": qt.text})
                return_dct[dct_key[-1]].update({qt.text: col_copy})
                counter += 1
    return return_dct

In [10]:
question_collector(soup)

{'General Info': {'General Info': {'Course': ['AAS 10 (A01) - Intro/African-American Studies'],
   'Instructor': ['Butler, Elizabeth Annette'],
   'Qtr/Yr': ['FA21'],
   'Enrollment': ['67'],
   'CAPEs Returned': ['40'],
   'Question': 'General Info'}},
 'PLEASE COMMENT ON THE FOLLOWING:': {'Your class level is': {'Freshman': ['10'],
   'Sophomore': ['16'],
   'Junior': ['7'],
   'Senior': ['7'],
   'Graduate': ['0'],
   'Extension': ['0'],
   'Visitor': ['0'],
   'Question': 'Your class level is'},
  'Your reason for taking this class is': {'Major': ['1'],
   'Minor': ['1'],
   'Gen. Ed.': ['29'],
   'Elective': ['6'],
   'Interest': ['3'],
   'Question': 'Your reason for taking this class is'},
  'What grade do you expect in this class?': {'A': ['26'],
   'B': ['8'],
   'C': ['2'],
   'D': ['0'],
   'F': ['0'],
   'P': ['4'],
   'NP': ['0'],
   'Question': 'What grade do you expect in this class?'}},
 'GENERAL QUESTIONS': {'I learned a great deal from this course.': {'Strongly Disagr

In [11]:
example_query = question_collector(soup)['PLEASE COMMENT ON THE FOLLOWING:']['Your class level is']
example_query

{'Freshman': ['10'],
 'Sophomore': ['16'],
 'Junior': ['7'],
 'Senior': ['7'],
 'Graduate': ['0'],
 'Extension': ['0'],
 'Visitor': ['0'],
 'Question': 'Your class level is'}

In [12]:
soup.find("span", {"id": "ContentPlaceHolder1_lblTermCode"}).text

'FA21'

In [13]:
test_df = {"Test": pd.DataFrame(data = example_query)}
#test_df["Test"].insert(0, "Course", ["AAS 10"])

In [14]:
test_df["Test"]

,Freshman,Sophomore,Junior,Senior,Graduate,Extension,Visitor,Question
0,10,16,7,7,0,0,0,Your class level is


In [15]:
len({'Question': 'Custom Question 5'})

1

In [16]:
def question_collector_to_pd(bs4obj):
    dict_collect = question_collector(bs4obj)
    return_dct = {}
    for key, value in dict_collect.items():
        for in_key, in_value in value.items():
            if len(value[in_key]) != 1:
                return_dct[in_key] = pd.DataFrame(data = in_value)
                return_dct[in_key].insert(0, "course", 
                                          bs4obj.find("span", {"id": "ContentPlaceHolder1_lblCourseDescription"})
                                          .text.split("(")[0].strip())
                return_dct[in_key].insert(0, "term", 
                                          bs4obj.find("span", {"id": "ContentPlaceHolder1_lblTermCode"}).text)
    return return_dct

In [20]:
pd_dct = question_collector_to_pd(soup)
pd_dct

{'General Info':    term  course                                         Course  \
 0  FA21  AAS 10  AAS 10 (A01) - Intro/African-American Studies   
 
                   Instructor Qtr/Yr Enrollment CAPEs Returned      Question  
 0  Butler, Elizabeth Annette   FA21         67             40  General Info  ,
 'Your class level is':    term  course Freshman Sophomore Junior Senior Graduate Extension Visitor  \
 0  FA21  AAS 10       10        16      7      7        0         0       0   
 
               Question  
 0  Your class level is  ,
 'Your reason for taking this class is':    term  course Major Minor Gen. Ed. Elective Interest  \
 0  FA21  AAS 10     1     1       29        6        3   
 
                                Question  
 0  Your reason for taking this class is  ,
 'What grade do you expect in this class?':    term  course   A  B  C  D  F  P NP                                 Question
 0  FA21  AAS 10  26  8  2  0  0  4  0  What grade do you expect in this class?,


In [17]:
len(list(pd_dct.values()))

26

In [18]:
list(pd_dct.values())

[   term  course                                         Course  \
 0  FA21  AAS 10  AAS 10 (A01) - Intro/African-American Studies   
 
                   Instructor Qtr/Yr Enrollment CAPEs Returned      Question  
 0  Butler, Elizabeth Annette   FA21         67             40  General Info  ,
    term  course Freshman Sophomore Junior Senior Graduate Extension Visitor  \
 0  FA21  AAS 10       10        16      7      7        0         0       0   
 
               Question  
 0  Your class level is  ,
    term  course Major Minor Gen. Ed. Elective Interest  \
 0  FA21  AAS 10     1     1       29        6        3   
 
                                Question  
 0  Your reason for taking this class is  ,
    term  course   A  B  C  D  F  P NP                                 Question
 0  FA21  AAS 10  26  8  2  0  0  4  0  What grade do you expect in this class?,
    term  course Strongly Disagree Disagree Neither Agree nor Disagree Agree  \
 0  FA21  AAS 10                 1        

In [29]:
df_return = pd.DataFrame([])
for values in pd_dct.values():
    df_return = pd.concat([df_return, values])
df_return = df_return.reset_index()

In [30]:
df_return

,index,term,course,Course,Instructor,Qtr/Yr,Enrollment,CAPEs Returned,Question,Freshman,...,12-13,14-15,16-17,18-19,20 or more,Very Rarely,Some of the Time,Most of the Time,Yes,No
0,0,FA21,AAS 10,AAS 10 (A01) - Intro/African-American Studies,"Butler, Elizabeth Annette",FA21,67,40,General Info,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,Your class level is,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,Your reason for taking this class is,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,What grade do you expect in this class?,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,I learned a great deal from this course.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,How many hours a week do you spend studying ou...,NaN,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
6,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,How often do you attend this course?,NaN,...,NaN,NaN,NaN,NaN,NaN,0,2,38,NaN,NaN
7,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,The course material is intellectually stimulat...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,Assignments promote learning.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,FA21,AAS 10,NaN,NaN,NaN,NaN,NaN,Required reading is useful.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
soup.find("span", {"id": "ContentPlaceHolder1_lblCourseDescription"}).text.split("(")[0].strip()

'AAS 10'

In [19]:
soup.find_all("span", {"id": 
                       re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_.*")})

[<span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_0">10<br/>25%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_1">16<br/>40%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_2">7<br/>18%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_3">7<br/>18%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_4">0<br/>0%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_5">0<br/>0%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_6">0<br/>0%<br/></span>]

In [20]:
re.search("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_", 
          "ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_0")

<re.Match object; span=(0, 51), match='ContentPlaceHolder1_dlQuestionnaire_rptChoiceText>

In [21]:
soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_.*")})

[<span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_0">PLEASE COMMENT ON THE FOLLOWING:</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_1">Your class level is</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_2">Your reason for taking this class is</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_3">What grade do you expect in this class?</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_4">GENERAL QUESTIONS</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_5">I learned a great deal from this course.</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_6">How many hours a week do you spend studying outside of class on average?</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_7">How often do you attend this course?</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_8">COURSE MATERIAL <b>AAS 10</b></span>,
 <spa